In [1]:
from pathlib import Path
from dotenv import load_dotenv
import os
import pandas as pd
from astropy import units as u
import numpy as np
from obsutils.utils import SkyportalClient
from galsynthspec.skyportal.query import strip_tns_name

In [2]:
# from galsynthspec.paths import data_dir as galsynthspec_data_dir

In [3]:
source = "AT2025abcr"
# source = "AT2025abao"

In [4]:
uvot_data_dir = Path(os.getenv("UVOT_DATA_DIR"))

In [5]:
send_to_fritz = True
instrument_id = 29 # UVOT is 46

In [6]:
df = pd.read_csv(uvot_data_dir / f"{source}/uvot_summary.csv")
# mask = uvot_df["AB_MAG"] > uvot_df["AB_MAG_LIM"]
# uvot_df.loc[mask, ["AB_MAG"]] = None
# uvot_df.loc[mask, ["AB_MAG_ERR"]] = None
df

,Unnamed: 0,ISOT,MJD,RA,DEC,FILTER,EXPOSURE,AB_MAG,AB_MAG_ERR,AB_MAG_LIM,PARENT_DIR
0,0,2025-11-09T16:36:04.308,60988.691717,26.730833,-15.371031,UVW2,676.944200,18.008690,0.048324,21.922050,3000183001
1,1,2025-11-09T18:17:39.345,60988.762261,26.730833,-15.371031,V,78.509880,17.610155,0.177369,18.563091,3000183001
2,2,2025-11-09T18:25:27.604,60988.767681,26.730833,-15.371031,UVM2,835.093300,18.362490,0.055101,22.218107,3000183001
3,3,2025-11-09T18:54:39.851,60988.787961,26.730833,-15.371031,UVW1,629.023900,18.372551,0.056614,21.375957,3000183001
4,4,2025-11-09T18:56:44.076,60988.789399,26.730833,-15.371031,U,314.083000,18.498495,0.063700,21.004894,3000183001
...,...,...,...,...,...,...,...,...,...,...,...
102,102,2026-01-04T13:05:56.141,61044.545789,26.730833,-15.371031,U,227.906750,19.211447,0.105205,20.892376,3000183020
103,103,2026-01-04T13:07:56.604,61044.547183,26.730833,-15.371031,B,141.296100,18.369953,0.124415,19.739655,3000183020
104,104,2026-01-04T13:53:52.158,61044.579076,26.730833,-15.371031,UVW2,357.040860,18.925833,0.075683,21.785349,3000183020
105,105,2026-01-04T13:57:43.805,61044.581757,26.730833,-15.371031,V,89.347090,17.817747,0.198743,18.595222,3000183020


In [7]:
mask = (df["AB_MAG_ERR"] < 1.5) & (df["AB_MAG"] < df["AB_MAG_LIM"])
df = df[mask]
df[["ISOT", "FILTER", "AB_MAG", "AB_MAG_ERR", "AB_MAG_LIM"]]

,ISOT,FILTER,AB_MAG,AB_MAG_ERR,AB_MAG_LIM
0,2025-11-09T16:36:04.308,UVW2,18.008690,0.048324,21.922050
1,2025-11-09T18:17:39.345,V,17.610155,0.177369,18.563091
2,2025-11-09T18:25:27.604,UVM2,18.362490,0.055101,22.218107
3,2025-11-09T18:54:39.851,UVW1,18.372551,0.056614,21.375957
4,2025-11-09T18:56:44.076,U,18.498495,0.063700,21.004894
...,...,...,...,...,...
102,2026-01-04T13:05:56.141,U,19.211447,0.105205,20.892376
103,2026-01-04T13:07:56.604,B,18.369953,0.124415,19.739655
104,2026-01-04T13:53:52.158,UVW2,18.925833,0.075683,21.785349
105,2026-01-04T13:57:43.805,V,17.817747,0.198743,18.595222


In [8]:
client = SkyportalClient()
client.set_up_session()

In [9]:
def get_ztf_name(source):
    res = client.api(
        "GET", 
        f"sources/{strip_tns_name(source)}",
    )
    res.raise_for_status()
    ztf_name = [x["obj_id"] for x in res.json()["data"]["duplicates"] if x["obj_id"][:3] == "ZTF"][0]
    return ztf_name


In [10]:
ztf_name = source if source[:3] == "ZTF" else get_ztf_name(source)
print(ztf_name)

ZTF25abxvhmk


In [11]:
ORIGIN = "uvotredux"

if send_to_fritz:

    payload = {
        "mjd": df["MJD"].to_list(),
        "filter": [f"uvot::{x.lower().strip()}" for x in df["FILTER"]],
        "obj_id": ztf_name,
        "instrument_id": instrument_id,
        "ra": df["RA"].to_list(),
        "dec": df["DEC"].to_list(),
        "ra_unc": None,
        "dec_unc": None,
        "origin": ORIGIN,
        "group_ids": [1],
        "magsys": "ab",
        "mag": df["AB_MAG"].fillna('').to_list(),
        "magerr": df["AB_MAG_ERR"].fillna('').to_list(),
        "limiting_mag": df["AB_MAG_LIM"].to_list(),
        "limiting_mag_nsigma": 3,
    }
    print(payload)
    res = client.api(
        "PUT", 
        "photometry",
        data=payload
    )
    print(res.json())
    res.raise_for_status()

{'mjd': [60988.69171652986, 60988.76226093275, 60988.76768059815, 60988.78796124155, 60988.78939903264, 60988.79019183206, 60993.22408534768, 60994.09180212164, 60996.76317907071, 60996.76397217083, 60996.76449648993, 60996.766392453705, 60996.76747161134, 60996.7699196044, 60999.5952762089, 60999.59668959271, 60999.59739294121, 60999.59932402118, 60999.6014550728, 60999.60396212477, 61003.17678676099, 61003.177957381486, 61003.17863627639, 61003.180827345255, 61003.18249122187, 61003.185491435535, 61005.730301341086, 61005.734527945024, 61005.7850333854, 61005.78692179363, 61005.78779957465, 61005.78893115127, 61007.95399700057, 61007.95483390024, 61007.95548431898, 61007.95813266655, 61007.95916157581, 61007.959503545135, 61011.78177075427, 61011.783100809145, 61011.783786535765, 61011.78576250093, 61011.78772933565, 61011.79023345093, 61014.561008505545, 61014.56255641771, 61014.563445691434, 61014.56638952697, 61014.56860779491, 61014.572832164005, 61017.51781998992, 61017.51986642